Importing Libraries:

In [417]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, classification_report

Functions:

In [418]:
def prepare_sampled_pixel_data(image, mask, sample_size=3000):
   
    # flatten image and mask
    h, w, c = image.shape
    X = image.reshape((h * w, c))  # rgb features
    y = mask.flatten()  # mask labels

    # ensures sample size does not exceed available data (an issue I encountered A LOT)
    sample_size = min(sample_size, len(y))
    indices = np.random.choice(len(y), size=sample_size, replace=False)
    return X[indices], y[indices]

def resize_mask(mask, image_shape):
  
    # resizes images to make sure it matches (fixed an issue I encountered A LOT)
    return cv2.resize(mask, (image_shape[1], image_shape[0]), interpolation=cv2.INTER_NEAREST)

def denoise_mask(mask):
    
    # denoises the mask so that we can keep our data normalized
    denoised = (mask > 127).astype(np.uint8)
    return denoised

# turned grayscale initially but lead to much lower accuracy, so turned it false
def load_images(paths, grayscale=False):

    # loads images from given paths
    images = []
    for path in paths:
        if not os.path.exists(path):
            print(f"Warning: File not found: {path}")
            continue
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_COLOR)
        if img is None:
            print(f"Warning: Failed to load image: {path}")
            continue
        images.append(img)
    if not images:
        raise FileNotFoundError("No valid images were loaded. Check file paths.")
    return images

Load Data:

In [419]:
# file Paths
train_image_paths = ["img1.png", "img2.png"]
train_mask_paths = ["mask1.png", "mask2.png"]
test_image_paths = ["unseen_img1.png", "unseen_img2.png"]
test_mask_paths = ["masked_img1.png", "masked_img2.png"]

# load and preprocess training data (resize and denoise)
train_images = load_images(train_image_paths)
train_masks = load_images(train_mask_paths, grayscale=True)
train_masks = [denoise_mask(resize_mask(msk, img.shape)) for img, msk in zip(train_images, train_masks)]

Prepare Data:

In [420]:
X_train, y_train = [], []
for img, msk in zip(train_images, train_masks):
    X, y = prepare_sampled_pixel_data(img, msk, sample_size=1500)
    X_train.append(X)
    y_train.append(y)

X_train = np.vstack(X_train) if X_train else np.empty((0, 4))  # Adjust for RGB + Texture
y_train = np.hstack(y_train) if y_train else np.empty((0,))
if X_train.size > 0 and y_train.size > 0:
    X_train, y_train = shuffle(X_train, y_train, random_state=42)

Train Model:

In [421]:
# Train the model
if X_train.size > 0 and y_train.size > 0:
    model = LogisticRegression(class_weight="balanced", max_iter=200)
    model.fit(X_train, y_train)
    print("Completed training")

# load and preprocess testing data
test_images = load_images(test_image_paths)
test_masks = load_images(test_mask_paths, grayscale=True)
test_masks = [denoise_mask(resize_mask(msk, img.shape)) for img, msk in zip(test_images, test_masks)]

# prepares and uses test set
X_test, y_test = [], []
for img, msk in zip(test_images, test_masks):
    X, y = prepare_sampled_pixel_data(img, msk, sample_size=1500)
    X_test.append(X)
    y_test.append(y)


Model training complete.


libpng warning: iCCP: known incorrect sRGB profile


Model Evaluation:

In [422]:
X_test = np.vstack(X_test)
y_test = np.hstack(y_test)

# model evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy: .2f}")
print("Classification report:")
print(classification_report(y_test, y_pred))


Accuracy on test set:  0.72
Classification report:
              precision    recall  f1-score   support

           0       0.82      0.84      0.83      2439
           1       0.23      0.21      0.22       561

    accuracy                           0.72      3000
   macro avg       0.53      0.53      0.53      3000
weighted avg       0.71      0.72      0.72      3000

